### Read Thai City

In [5]:
#select ชื่อเมืองในไทย
import json

with open('city.list.json', 'r') as f:
    d_test = json.load(f)

th_cities = [city for city in d_test if city['country'] == 'TH']
th_cities[:5]

#count = len(th_cities)
#count

FileNotFoundError: [Errno 2] No such file or directory: 'city.list.json'

In [10]:
provinces = {
    city['name']: {
        'lat': city['coord']['lat'],
        'lon': city['coord']['lon']
    } for city in th_cities
}
# provinces
dict(list(provinces.items())[:5])

{'Don Sak': {'lat': 9.31676, 'lon': 99.691841},
 'Ban Thung That': {'lat': 8.26667, 'lon': 99.583328},
 'Thung Sai': {'lat': 16.31489, 'lon': 99.832672},
 'Tham Phannara': {'lat': 8.42044, 'lon': 99.395172},
 'Ban Talat Yai': {'lat': 7.88481, 'lon': 98.400078}}

### Fetch weather Data

In [ ]:
# Test Find Lat Lon
province = "Pathum Thani"
print(
    provinces[province]['lat'],
    provinces[province]['lon']
)

14.01346 100.530487


In [ ]:
#Endpoint API
WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "68c16da5e675ad8635df84629765b118"

In [ ]:
import requests
import pandas as pd
from datetime import datetime
import time
import pytz

# # Endpoint API
# WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
# API_KEY = "68c16da5e675ad8635df84629765b118"


# Function to fetch and process weather data

def get_weather_data(province='Pathum Thani'):
    #Sleep
    # time.sleep(2)
    #ENDPOINT API Parameters
    WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
    API_KEY = "68c16da5e675ad8635df84629765b118"
    params = {
        "lat": provinces[province]['lat'],
        "lon": provinces[province]['lon'],
        "appid": API_KEY,
        "units": "metric"
    }

    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        # return data # Fetch Weather Data
        
        # Convert timestamp to datetime
        dt = datetime.now()
        thai_tz = pytz.timezone('Asia/Bangkok')
        created_at = dt.replace(tzinfo=thai_tz)
        
        timestamp = datetime.now()
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'created_at': created_at,
            'requested_province' : province,
            'location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None
    

In [15]:
get_weather_data()

{'timestamp': datetime.datetime(2025, 4, 26, 2, 8, 43, 669855),
 'year': 2025,
 'month': 4,
 'day': 26,
 'hour': 2,
 'minute': 8,
 'created_at': datetime.datetime(2025, 4, 26, 2, 8, 43, 628633, tzinfo=<DstTzInfo 'Asia/Bangkok' LMT+6:42:00 STD>),
 'requested_province': 'Pathum Thani',
 'location': 'Pathum Thani',
 'weather_main': 'Clouds',
 'weather_description': 'few clouds',
 'main.temp': 29.99}

### FLOW

In [23]:
weatherdata_df = pd.DataFrame([get_weather_data(p) for p in list(provinces.keys())[:2]])
weatherdata_df

,timestamp,year,month,day,hour,minute,created_at,requested_province,location,weather_main,weather_description,main.temp
0,2025-04-26 02:17:25.524864,2025,4,26,2,17,2025-04-26 02:35:25.524837+07:00,Don Sak,Don Sak,Clouds,overcast clouds,27.17
1,2025-04-26 02:17:25.641887,2025,4,26,2,17,2025-04-26 02:35:25.641859+07:00,Ban Thung That,Ban Thung That,Clear,clear sky,24.42


### TEST

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("OPENWEATHER_API_KEY")
ACCESS_KEY = os.getenv("LAKEFS_ACCESS_KEY")
SECRET_KEY = os.getenv("LAKEFS_SECRET_KEY")
lakefs_endpoint = os.getenv("LAKEFS_ENDPOINT", "http://lakefs-dev:8000")
print(API_KEY)
print(ACCESS_KEY)
print(SECRET_KEY)
print(lakefs_endpoint)

13fd31c548dbeae77a5c0d773a99df79
access_key
secret_key
http://lakefs-dev:8000/


In [4]:
import pandas as pd
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  #/home/jovyan/work
coord_path = os.path.join(BASE_DIR, "save", "district_coord.csv")
coord_df = pd.read_csv(coord_path)
df_sample = coord_df.sample(10)
df_sample

,province_id,province_th,province_en,district_id,district_th,district_en,lat,lon
687,17,สิงห์บุรี,Singburi,1701,เมืองสิงห์บุรี,Mueang Sing Buri,14.899800,100.405900
689,72,สุพรรณบุรี,Suphanburi,7201,เมืองสุพรรณบุรี,Mueang Suphan Buri,14.474180,100.122180
407,14,พระนครศรีอยุธยา,Phra Nakhon Si Ayutthaya,1416,บ้านแพรก,Ban Phraek,14.647700,100.582500
922,58,แม่ฮ่องสอน,Mae Hong Son,5805,แม่ลาน้อย,Mae La Noi,18.384444,97.936944
766,41,อุดรธานี,Udon Thani,4103,หนองวัวซอ,Nong Wua So,17.162500,102.572700
494,95,ยะลา,Yala,9501,เมืองยะลา,Mueang Yala,6.546371,101.284322
912,54,แพร่,Phrae,5406,สอง,Song,18.576900,100.215200
37,10,กรุงเทพมหานคร,Bangkok,1003,หนองจอก,Nong Khaem,13.856300,100.863000
533,45,ร้อยเอ็ด,Roi Et,4520,ทุ่งเขาหลวง,Thung Khao Luang,15.989660,103.870860
345,38,บึงกาฬ,Bueng Kan,3806,บึงโขงหลง,Bueng Khong Long,18.008990,104.073020


In [11]:
import os
import json

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  #/home/jovyan/work

def load_geojson(geojson_path):
    with open(geojson_path, "r", encoding="utf-8") as f:
        return json.load(f)

province_geojson_path = os.path.join(BASE_DIR, "save", "gadm41_THA_1.json")
province_geojson = load_geojson(province_geojson_path)
# province_geojson["features"][:2]

In [20]:
import s3fs
import pyarrow.dataset as ds
import os
import pandas as pd

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
coord_path = os.path.join(BASE_DIR, "save", "district_coord.csv")
data_path = os.path.join(BASE_DIR, "save", "bd5d4217b47743f2b597ac5cd8293ba0-0.parquet")

df_code = pd.read_csv(coord_path)
df = pd.read_parquet(data_path)

df_code = df_code.rename(columns={"province_en" : "province" ,"district_en":"district"})
df = pd.merge(
    df,
    df_code[["province", "district", "district_id"]],
    on=["district", "province"],
    how="left"  # ใช้ 'left' เพื่อคงข้อมูล df หลักไว้ทั้งหมด
)
# pollution_path = 'pollution-data/main/pollution.parquet'
# df = load_data(pollution_path)
df.head()
# df.info()

,timestamp,minute,localtime,province,district,lat,lon,main.aqi,components_co,components_no,components_no2,components_o3,components_so2,components_pm2_5,components_pm10,components_nh3,district_id
0,2025-05-11 17:53:29.373852,53,2025-05-12 00:53:29.373852+07:00,Krabi,Nuea Khlong,8.0747,99.0036,1,103.96,0.0,0.29,45.12,0.09,1.35,4.62,0.00,8108
1,2025-05-11 17:53:29.388147,53,2025-05-12 00:53:29.388147+07:00,Krabi,Lam Thap,8.0717,99.2917,1,104.72,0.0,0.30,44.72,0.08,1.22,3.89,0.00,8107
2,2025-05-11 17:53:29.181085,53,2025-05-12 00:53:29.181085+07:00,Krabi,Plai Phraya,8.5333,98.8628,1,108.50,0.0,0.39,47.41,0.10,1.24,4.16,0.01,8106
3,2025-05-11 17:53:29.190713,53,2025-05-12 00:53:29.190713+07:00,Krabi,Khao Phanom,8.2647,99.0492,1,106.81,0.0,0.29,47.46,0.06,1.09,3.26,0.00,8102
4,2025-05-11 17:53:29.361694,53,2025-05-12 00:53:29.361694+07:00,Krabi,Ao Luek,8.3775,98.7217,1,105.78,0.0,0.34,47.24,0.11,1.56,6.17,0.00,8105


In [35]:
import geopandas as gpd

# โหลด geojson
# gdf = gpd.read_file(province_geojson_path)
import os
import json

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))  #/home/jovyan/work

province_geojson_path = os.path.join(BASE_DIR, "save", "gadm41_THA_2.json")
gdf = gpd.read_file(province_geojson_path)

# สร้าง dict mapping มือ
district_id_map = {
    ("BuengKan", "BungKan"): "3801",
    ("BuengKan", "BungKhongLong"): "3806",
    ("BuengKan", "K.BungKhla"): "3808",
    ("BuengKan", "PakKhat"): "3805",
    ("BuengKan", "PhonCharoen"): "3802",
    ("BuengKan", "Seka"): "3804",
    ("BuengKan", "SiWilai"): "3807",
    ("BuengKan", "SoPhisai"): "3803",
    ("Chanthaburi", "MuangChanthaburi"): "2201",
    ("KhonKaen", "WiangKao"): "4029"
    # 10:"NA - ("Phatthalung" - SongkhlaLake"
    # 11:"NA - ("Songkhla" - SongkhlaLake"
}

# เติม district_id ใหม่เข้า geojson
def map_district_id(row):
    return district_id_map.get((row["NAME_1"], row["NAME_2"]), None)

gdf["CC_2"] = gdf.apply(map_district_id, axis=1)


# ตรวจว่ามีอันไหนไม่เจอ
print("Missing:", gdf["CC_2"].isnull().sum())
gdf["CC_2"]

print(gdf[gdf["CC_2"].notnull()][["NAME_1", "NAME_2", "CC_2"]])

# บันทึกกลับ
# gdf.to_file("districts_updated.geojson", driver='GeoJSON')

Missing: 918
          NAME_1            NAME_2  CC_2
64      BuengKan           BungKan  3801
65      BuengKan     BungKhongLong  3806
66      BuengKan        K.BungKhla  3808
67      BuengKan           PakKhat  3805
68      BuengKan       PhonCharoen  3802
69      BuengKan              Seka  3804
70      BuengKan           SiWilai  3807
71      BuengKan          SoPhisai  3803
136  Chanthaburi  MuangChanthaburi  2201
268     KhonKaen          WiangKao  4029


In [25]:
import geopandas as gpd
import os

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
geojson_path = os.path.join(BASE_DIR, "save", "gadm41_THA_2.json")
gdf = gpd.read_file(geojson_path)


district_id_map = {
    ("BuengKan", "BungKan"): "3801",
    ("BuengKan", "BungKhongLong"): "3806",
    ("BuengKan", "K.BungKhla"): "3808",
    ("BuengKan", "PakKhat"): "3805",
    ("BuengKan", "PhonCharoen"): "3802",
    ("BuengKan", "Seka"): "3804",
    ("BuengKan", "SiWilai"): "3807",
    ("BuengKan", "SoPhisai"): "3803",
    ("Chanthaburi", "MuangChanthaburi"): "2201",
    ("KhonKaen", "WiangKao"): "4029"
    # เพิ่มได้เรื่อยๆ
}

def map_district_id(row):
    return district_id_map.get((row["NAME_1"], row["NAME_2"]), None)

gdf["district_id"] = gdf.apply(map_district_id, axis=1)

changed = gdf[gdf["district_id"].notna()]

# print("อำเภอที่ถูกอัปเดต:", changed[["NAME_1", "NAME_2", "CC_2"]])

# เอาค่าใหม่ (district_id จาก mapping) ไปแทนที่ค่าใน CC_2 เฉพาะแถวที่ไม่ใช่ None
gdf["CC_2"] = gdf["district_id"].combine_first(gdf["CC_2"])

# จากนั้นลบ column ชั่วคราว
gdf_cleaned = gdf.drop(columns=["district_id"])
gdf_cleaned[gdf_cleaned["NAME_1"] == "BuengKan"]

# gdf_cleaned.to_file("gadm41_THA_2_clean.geojson", driver="GeoJSON")


In [22]:
import geopandas as gpd
import os

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
geojson_path = os.path.join(BASE_DIR, "save", "gadm41_THA_1.json")
gdf = gpd.read_file(geojson_path)


province_id_map = {
    ("BuengKan"): "38",
    ("Chanthaburi"): "22",
    # เพิ่มได้เรื่อยๆ
}

def map_province_id(row):
    return province_id_map.get((row["NAME_1"]), None)

gdf["province_id"] = gdf.apply(map_province_id, axis=1)

changed = gdf[gdf["province_id"].notna()]

# print("อำเภอที่ถูกอัปเดต:", changed[["NAME_1", "NAME_2", "CC_2"]])

# เอาค่าใหม่ (district_id จาก mapping) ไปแทนที่ค่าใน CC_2 เฉพาะแถวที่ไม่ใช่ None
gdf["CC_1"] = gdf["province_id"].combine_first(gdf["CC_1"])

# จากนั้นลบ column ชั่วคราว
gdf_cleaned = gdf.drop(columns=["province_id"])
# gdf_cleaned.sort_values(by="CC_1")
# gdf_cleaned["CC_1"].unique()

# gdf_cleaned["CC_1"].isna().mean() * 100  # คิดเป็น %
# gdf_cleaned.to_file("gadm41_THA_1_clean.geojson", driver="GeoJSON")


In [26]:
pip freeze

aiobotocore==2.22.0
aiohappyeyeballs==2.6.1
aiohttp==3.11.18
aioitertools==0.12.0
aiosignal==1.3.2
aiosqlite==0.20.0
alembic==1.13.2
altair==5.5.0
annotated-types==0.7.0
antlr4-python3-runtime==4.9.3
anyio==4.4.0
anywidget==0.9.18
appdirs==1.4.4
apprise==1.8.1
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asgi-lifespan==2.1.0
asttokens==3.0.0
async-lru==2.0.5
async-timeout==4.0.3
asyncpg==0.29.0
attrs==23.2.0
babel==2.17.0
beautifulsoup4==4.13.4
binaryornot==0.4.4
bleach==6.2.0
blinker==1.9.0
botocore==1.37.3
bqplot==0.12.44
branca==0.8.1
build==1.2.2.post1
cachetools==5.4.0
certifi==2024.7.4
cffi==1.16.0
chardet==5.2.0
charset-normalizer==3.3.2
click==8.1.7
cloudpickle==3.0.0
colorama==0.4.6
colour==0.1.5
comm==0.2.2
contourpy==1.3.2
cookiecutter==2.6.0
coolname==2.2.0
croniter==3.0.3
cryptography==43.0.0
cycler==0.12.1
dateparser==1.2.0
debugpy==1.8.14
decorator==5.2.1
defusedxml==0.7.1
dnspython==2.6.1
docker==7.1.0
duckdb==1.2.2
dynaconf==3.2.11
email_validator==2.2